In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

# Preparamos los Sets de Training

In [2]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets"

In [3]:
ins_18_20 = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); 
ins_21_23 = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); 
ins_24_26 = pd.read_csv( loc_ts + "\\ins_24_26.csv" ); 

In [4]:
evt_18_20 = pd.read_csv( loc_ts + "\\evt_18_20.csv" );
evt_21_23 = pd.read_csv( loc_ts + "\\evt_21_23.csv" );
evt_24_26 = pd.read_csv( loc_ts + "\\evt_24_26.csv" );

In [5]:
auc_18_20 = pd.read_csv( loc_ts + "\\auc_18_20.csv" ); 
auc_21_23 = pd.read_csv( loc_ts + "\\auc_21_23.csv" );
auc_24_26 = pd.read_csv( loc_ts + "\\auc_24_26.csv" );

In [6]:
clk_18_20 = pd.read_csv( loc_ts + "\\clk_18_20.csv" );
clk_21_23 = pd.read_csv( loc_ts + "\\clk_21_23.csv" );
clk_24_26 = pd.read_csv( loc_ts + "\\clk_24_26.csv" );

# Genero los Sets con el primer install

In [7]:
auc_18_20["date"] =  pd.to_datetime( auc_18_20["date"] ); auc_18_20["_st"] = auc_18_20["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); auc_18_20["_st"] = auc_18_20["_st"].dt.total_seconds();
auc_21_23["date"] =  pd.to_datetime( auc_21_23["date"] ); auc_21_23["_st"] = auc_21_23["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); auc_21_23["_st"] = auc_21_23["_st"].dt.total_seconds(); 
auc_24_26["date"] =  pd.to_datetime( auc_24_26["date"] ); auc_24_26["_st"] = auc_24_26["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); auc_24_26["_st"] = auc_24_26["_st"].dt.total_seconds(); 

In [ ]:
auc_18_20_first_auc = auc_18_20.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_21_23_first_auc = auc_21_23.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_24_26_first_auc = auc_24_26.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )

# Sacamos los Features

In [ ]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_auc"

In [ ]:
rh_18_20 = auc_18_20_first_auc[ ["ref_hash"] ] #RH unicos que coinciden con los datos que usamos para entrenar
rh_21_23 = auc_21_23_first_auc[ ["ref_hash"] ] #RH unicos que coinciden con los datos que queremos predecir

tg_18_20 = auc_18_20_first_auc[ ["_st"] ]
tg_21_23 = auc_21_23_first_auc[ ["_st"] ]

## <span style="color:green"> **Hora de la realizacion de la primera subasta** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. Por ejemplo que las 00 hs tengan un valor similar a 2

In [51]:
hora_auc_18_20 = rh_18_20.copy()
hora_auc_21_23 = rh_21_23.copy()

In [52]:
hora_auc_18_20["hour"] = auc_18_20_first_auc["date"].dt.hour; hora_auc_18_20["hour"] = hora_auc_18_20["hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
hora_auc_21_23["hour"] = auc_21_23_first_auc["date"].dt.hour; hora_auc_21_23["hour"] = hora_auc_21_23["hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

In [53]:
hora_auc_18_20.to_csv( loc_ftr + "\\hora_auc_18_20.csv", index = False )
hora_auc_21_23.to_csv( loc_ftr + "\\hora_auc_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de Clicks en la ventana anterior** </span> 

In [66]:
cant_clk_18_20 = rh_18_20.copy()
cant_clk_21_23 = rh_21_23.copy()

In [67]:
cant_clk_18_20 = cant_clk_18_20.merge( clk_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_clk_21_23 = cant_clk_21_23.merge( clk_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

In [68]:
cant_clk_18_20.fillna( 0, inplace = True )
cant_clk_21_23.fillna( 0, inplace = True )

In [69]:
cant_clk_18_20.to_csv( loc_ftr + "\\cant_clk_18_20.csv", index = False )
cant_clk_21_23.to_csv( loc_ftr + "\\cant_clk_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones por dispositivo en la ventana previa a la conversion** </span> 

In [70]:
cant_ins_18_20 = rh_18_20.copy()
cant_ins_21_23 = rh_21_23.copy()

In [71]:
cant_ins_18_20 = cant_ins_18_20.merge( ins_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_ins_21_23 = cant_ins_21_23.merge( ins_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

In [72]:
cant_ins_18_20.fillna( 0, inplace = True )
cant_ins_21_23.fillna( 0, inplace = True )

In [73]:
cant_ins_18_20.to_csv( loc_ftr + "\\cant_ins_18_20.csv", index = False )
cant_ins_21_23.to_csv( loc_ftr + "\\cant_ins_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de subastas por dispositivo en la ventana previa a la conversion** </span> 

In [83]:
cant_auc_18_20 = rh_18_20.copy()
cant_auc_21_23 = rh_21_23.copy()

In [84]:
cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

In [85]:
cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

In [86]:
cant_auc_18_20.to_csv( loc_ftr + "\\cant_auc_18_20.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\cant_auc_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de eventos por dispositivo en la ventana previa a la conversion** </span> 

In [89]:
cant_evt_18_20 = rh_18_20.copy()
cant_evt_21_23 = rh_21_23.copy()

In [90]:
cant_evt_18_20 = cant_evt_18_20.merge( evt_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_evt_21_23 = cant_evt_21_23.merge( evt_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

In [91]:
cant_evt_18_20.fillna( 0, inplace = True )
cant_evt_21_23.fillna( 0, inplace = True )

In [92]:
cant_evt_18_20.to_csv( loc_ftr + "\\cant_evt_18_20.csv", index = False )
cant_evt_21_23.to_csv( loc_ftr + "\\cant_evt_21_23.csv", index = False )

## <span style="color:green"> **Ref Type del dispositivo que realizo la instalacion** </span> 

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [95]:
ref_type_18_20 = rh_18_20.copy()
ref_type_21_23 = rh_21_23.copy()

In [98]:
rt_ohe_18_20 = pd.get_dummies( auc_18_20_first_auc["ref_type_id"] )
rt_ohe_21_23 = pd.get_dummies( auc_21_23_first_auc["ref_type_id"] )

In [99]:
ref_type_18_20 = pd.concat( [ref_type_18_20, rt_ohe_18_20], axis = 1 )
ref_type_21_23 = pd.concat( [ref_type_21_23, rt_ohe_21_23], axis = 1 )

In [102]:
ref_type_18_20.to_csv( loc_ftr + "\\ref_type_18_20.csv", index = False )
ref_type_21_23.to_csv( loc_ftr + "\\ref_type_21_23.csv", index = False )

## <span style="color:green"> **Dia de la semana realizacion de la primera subasta** </span> 

Le aplicamos la transformacion sin( (hora * pi)/6 ) para agregarle periodicidad. Por ejemplo 0 (lunes) tengan un valor similar a 6 (domingo)

In [106]:
sdia_auc_18_20 = rh_18_20.copy()
sdia_auc_21_23 = rh_21_23.copy()

In [107]:
sdia_auc_18_20["hour"] = auc_18_20_first_auc["date"].dt.hour; sdia_auc_18_20["hour"] = sdia_auc_18_20["hour"].apply( lambda x: np.sin( (x*np.pi)/6 ) )
sdia_auc_21_23["hour"] = auc_21_23_first_auc["date"].dt.hour; sdia_auc_21_23["hour"] = sdia_auc_21_23["hour"].apply( lambda x: np.sin( (x*np.pi)/6 ) )

In [108]:
sdia_auc_18_20.to_csv( loc_ftr + "\\sdia_auc_18_20.csv", index = False )
sdia_auc_21_23.to_csv( loc_ftr + "\\sdia_auc_21_23.csv", index = False )

## <span style="color:green"> **Source que recibio mas instalaciones por dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la fuente principal de algun dispositivo)

In [109]:
srce_auc_18_20 = rh_18_20.copy()
srce_auc_21_23 = rh_21_23.copy()

In [110]:
main_source_18_20 = auc_18_20.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_21_23 = auc_21_23.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

In [111]:
main_source_18_20 = main_source_18_20.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_18_20["cant_auc"] )
main_source_21_23 = main_source_21_23.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_21_23["cant_auc"] )

In [112]:
srce_auc_18_20 = srce_auc_18_20.merge( main_source_18_20.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_18_20.fillna( "no_font", inplace = True )
srce_auc_21_23 = srce_auc_21_23.merge( main_source_21_23.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_21_23.fillna( "no_font", inplace = True )

In [113]:
srce_auc_18_20["to_count"] = 1; srce_auc_18_20["source_id"] = srce_auc_18_20[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_18_20); del(srce_auc_18_20["to_count"])
srce_auc_21_23["to_count"] = 1; srce_auc_21_23["source_id"] = srce_auc_21_23[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_21_23); del(srce_auc_21_23["to_count"])

In [114]:
srce_auc_18_20.to_csv( loc_ftr + "\\srce_auc_18_20.csv", index = False )
srce_auc_21_23.to_csv( loc_ftr + "\\srce_auc_21_23.csv", index = False )

## <span style="color:green"> **Ref Hash** </span> 


#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de apariciones en las subastas en la ventana anterior)

In [47]:
rh_encod_18_20 = rh_18_20.copy()
rh_encod_21_23 = rh_21_23.copy()

In [47]:
appces_18_20_auc = auc_18_20["device_id"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "device_id":"appearances"} )
appces_21_23_auc = auc_21_23["device_id"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "device_id":"appearances"} )

In [47]:
appces_18_20_auc["ref_hash_mean"] = appces_18_20_auc["appearances"] / len( auc_18_20 ); appces_18_20_auc.drop( "appearances", axis = 1, inplace = True )
appces_21_23_auc["ref_hash_mean"] = appces_21_23_auc["appearances"] / len( auc_21_23 ); appces_21_23_auc.drop( "appearances", axis = 1, inplace = True )

In [47]:
rh_encod_18_20 = rh_encod_18_20.merge( appces_18_20_auc, how = "left", on = "ref_hash" )
rh_encod_21_23 = rh_encod_21_23.merge( appces_21_23_auc, how = "left", on = "ref_hash" )

In [47]:
rh_encod_18_20.to_csv( loc_ftr + "\\rh_encod_18_20.csv", index = False )
rh_encod_21_23.to_csv( loc_ftr + "\\rh_encod_21_23.csv", index = False )

## <span style="color:green"> **Hora mas popular del usuario en las subastas** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [99]:
main_ahr_18_20 = rh_18_20.copy()
main_ahr_21_23 = rh_21_23.copy()

In [99]:
hour_mode_18_20 = auc_18_20[["device_id"]]; hour_mode_18_20["hour"] = pd.to_datetime( auc_18_20["date"] ).dt.hour
hour_mode_21_23 = auc_21_23[["device_id"]]; hour_mode_21_23["hour"] = pd.to_datetime( auc_21_23["date"] ).dt.hour

In [99]:
hour_mode_18_20 = hour_mode_18_20.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_21_23 = hour_mode_21_23.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

In [99]:
hour_mode_18_20 = hour_mode_18_20.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)
hour_mode_21_23 = hour_mode_21_23.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)

In [99]:
main_ahr_18_20 = main_ahr_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ahr_21_23 = main_ahr_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

In [99]:
main_ahr_18_20["main_hour"] =  main_ahr_18_20["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ahr_21_23["main_hour"] =  main_ahr_21_23["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

In [99]:
main_ahr_18_20.to_csv( loc_ftr + "\\main_ahr_18_20.csv", index = False )
main_ahr_21_23.to_csv( loc_ftr + "\\main_ahr_21_23.csv", index = False )